# Introduction to OGGM

This tutorial is a first introduction into running the Open Global Glacier Model OGGM and is based on [this notebook](https://tutorials.oggm.org/stable/notebooks/10minutes/preprocessed_directories.html) from the OGGM tutorials. The OGGM workflow is best explained with an example. In the following, we will show you the OGGM fundamentals ([Doc page: model structure and fundamentals](https://docs.oggm.org/en/stable/structure.html)). This example is also meant to guide you through a first-time setup if you are using OGGM on your own computer.

## First time set-up 

### Input data folders

**If you are using your own computer**: before you start, make sure that you have set-up the [input data configuration file](https://docs.oggm.org/en/stable/getting-started.html#system-settings) at your wish.

In the course of this tutorial, we will need to download data needed for each glacier (a few 100s of mb at max, depending on the chosen glaciers), so make sure you have an internet connection.

### cfg.initialize() and cfg.PARAMS 

An OGGM simulation script will always start with the following commands:

In [ ]:
from oggm import cfg, utils
cfg.initialize()

A call to [cfg.initialize()](https://docs.oggm.org/en/stable/generated/oggm.cfg.initialize.html) will read the default parameter file (or any user-provided file) and make them available to all other OGGM tools via the `cfg.PARAMS` dictionary. Here are some examples of these parameters:

In [ ]:
cfg.PARAMS['melt_f'], cfg.PARAMS['ice_density'], cfg.PARAMS['continue_on_error']

See [here](https://github.com/OGGM/oggm/blob/stable/oggm/params.cfg) for the default parameter file and a description of their role and default value.

In [ ]:
# You can try with or without multiprocessing: with two glaciers, OGGM could run on two processors
cfg.PARAMS['use_multiprocessing'] = True

## Workflow

In this section, we will explain the fundamental concepts of the OGGM workflow:
- Working directories
- Glacier directories
- Tasks

### Working directory 

Each OGGM run needs a **single folder** where to store the results of the computations for all glaciers. This is called a "working directory" and needs to be specified before each run. Here we create a persistent OGGM working directory, to have a look at the individual files later.

In [ ]:
# define the path
path = 'intro_working_dir'

# create the directory, if reset=True ALL EXISTING DATA will be deleted
utils.mkdir(path, reset=False)

# tell OGGM where to work
cfg.PATHS['working_dir'] = path
cfg.PATHS['working_dir']

If you just want to play around with the model without saving any data for later, you can create a [temporary directory](https://en.wikipedia.org/wiki/Temporary_folder), using `utils.gettempdir(dirname='OGGM-GettingStarted', reset=True)`. However, in practice for a real simulation, you should set the working directory yourself, just like in the cell above. Note: The size of this directory will depend on how many glaciers you'll simulate!

<div class="alert alert-info">
    <b>
        In the OGGM design, this working directory is meant to be persistent, at least as long as you need the data. For example, you can stop your processing workflow after any task, and restart from an existing working directory at a later stage, simply by using the same working directory.
    </b>
</div>

### Define the glaciers for the run 

In [ ]:
rgi_ids = ['RGI60-11.01328', 'RGI60-11.01450'] 

You can provide any number of glacier identifiers to OGGM. In this case, we chose: 
- `RGI60-11.01328`: [Unteraar Glacier](https://en.wikipedia.org/wiki/Unteraargletscher) in the Swiss Alps
- `RGI60-11.01450`: [Aletsch Glacier](https://en.wikipedia.org/wiki/Aletsch_Glacier) in the Swiss Alps

Here is a list of other glaciers you might want to try out:
- `RGI60-11.00897`: [Hintereisferner](http://acinn.uibk.ac.at/research/ice-and-climate/projects/hintereisferner) in the Austrian Alps.
- `RGI60-18.02342`: [Tasman Glacier](https://en.wikipedia.org/wiki/Tasman_Glacier) in New Zealand
- `RGI60-11.00787`: [Kesselwandferner](https://de.wikipedia.org/wiki/Kesselwandferner) in the Austrian Alps
- ... or any other glacier identifier! You can find other glacier identifiers by exploring the [GLIMS viewer](https://www.glims.org/maps/glims). See the [working with the RGI](https://tutorials.oggm.org/stable/notebooks/tutorials/working_with_rgi.html) OGGM tutorial for an introduction on RGI IDs and the GLIMS browser.


<div class="alert alert-warning">
    <b>Questions</b>: Use your favourite search engine to try and answer the following questions:
        <ul>
            <li>What is the Randolph Glacier Inventory RGI?</li>
            <li>OGGM uses RGI6 and not RGI7. Could you guess why?</li>
            <li>Use the <a href='https://www.glims.org/maps/glims'>GLIMS viewer</a> to find one RGI7 ID that corresponds to one RGI6 ID for one of the glaciers listed above.</li>
        </ul>
</div>

### Glacier directories 

The OGGM workflow is organized as a list of **tasks** that have to be applied to a list of glaciers. The vast majority of tasks are called **entity tasks**: they are standalone operations to be realized on one single glacier entity. These tasks are executed sequentially (one after another): they often need input generated by the previous task(s): for example, the climate calibration needs the glacier flowlines, which can be only computed after the topography data has been processed, and so on.

To handle this situation, OGGM uses a workflow based on data persistence on disk: instead of passing data as python variables from one task to another, each task will read the data from disk and then write the computation results back to the disk, making these new data available for the next task in the queue. These glacier specific data are located in [glacier directories](https://docs.oggm.org/en/stable/generated/oggm.GlacierDirectory.html#oggm.GlacierDirectory). 

One main advantage of this workflow is that OGGM can prepare data and make it available to everyone! Here is an example of an url where such data can be found:

In [ ]:
from oggm import workflow
from oggm import DEFAULT_BASE_URL
DEFAULT_BASE_URL

Let's use OGGM to download the glacier directories for our selected glaciers:

In [ ]:
gdirs = workflow.init_glacier_directories(
    rgi_ids,  # which glaciers?
    prepro_base_url=DEFAULT_BASE_URL,  # where to fetch the data?
    from_prepro_level=4,  # what kind of data? 
    prepro_border=80  # how big of a map?
)

- the keyword `from_prepro_level` indicates that we will start from [pre-processed directories](https://docs.oggm.org/en/stable/shop.html#pre-processed-directories), i.e. data that are already prepared by the OGGM team. In many cases you will want to start from pre-processed directories, in most cases from level 3 or 5. For level 3 and above the model has already been calibrated, so you no longer need to do that yourself and can start rigth away with your simulation. Here we start from level 4 and add some data to the processing in order to demonstrate the OGGM workflow.
- the `prepro_border` keyword indicates the number of grid points which we'd like to add to each side of the glacier for the local map: the larger the glacier will grow, the larger the border parameter should be. The available pre-processed border values are: **10, 80, 160, 240** (depending on the model set-ups there might be more or less options). These are the fixed map sizes we prepared for you - any other map size will require a full processing (see the [further DEM sources example](https://tutorials.oggm.org/stable/notebooks/tutorials/dem_sources.html) for a tutorial).

The [init_glacier_directories](https://docs.oggm.org/en/stable/generated/oggm.workflow.init_glacier_directories.html#oggm.workflow.init_glacier_directories) task will allways be the very first task to call for all your OGGM experiments. Let's see what it gives us back:

In [ ]:
type(gdirs), type(gdirs[0])

`gdirs` is a list of [GlacierDirectory](https://docs.oggm.org/en/stable/generated/oggm.GlacierDirectory.html#oggm.GlacierDirectory) objects (one for each glacier). **Glacier directories are used by OGGM as "file and attribute manager"** for single glaciers. For example, the model now knows where to find the topography data file for this glacier:

In [ ]:
gdir = gdirs[0]  # take the first glacier
print('Path to the DEM:', gdir.get_filepath('dem'))

And we can also access some attributes of this glacier:

In [ ]:
gdir

In [ ]:
gdir.rgi_date  # date at which the outlines are valid

The advantage of this Glacier Directory data model is that it simplifies greatly the data transfer between tasks. **The single mandatory argument of most OGGM commands will allways be a glacier directory**. With the glacier directory, each OGGM task will find the input it needs: for example, both the glacier's topography and outlines are needed for the next plotting function, and both are available via the `gdir` argument:

In [ ]:
from oggm import graphics
graphics.plot_domain(gdir, figsize=(6, 5))

Another advantage of glacier directories is their persistence on disk: once created, **they can be recovered from the same location** by using `init_glacier_directories` again, but without keyword arguments:

In [ ]:
# Fetch the LOCAL pre-processed directories - note that no arguments are used!
gdirs = workflow.init_glacier_directories(rgi_ids)

See the [store_and_compress_glacierdirs](https://tutorials.oggm.org/stable/notebooks/tutorials/store_and_compress_glacierdirs.html) tutorial for more information on glacier directories and how to use them for operational workflows.

### Accessing data in the preprocessed directories

Glacier directories are the central object for model users and developpers to access data for this glacier. Let's say for example that you would like to retrieve the climate data that we have prepared for you. You can ask the glacier directory to tell you where this data is:

In [ ]:
gdir.get_filepath('climate_historical')

One can then use familiar tools to read and process the data further:

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
# Open the file using xarray
with xr.open_dataset(gdir.get_filepath('climate_historical')) as ds:
    ds = ds.load()
# Plot the data
ds.temp.resample(time='YS').mean().plot(label=f'Annual temperature at {int(ds.ref_hgt)}m a.s.l.');
ds.temp.resample(time='YS').mean().rolling(time=31, center=True, min_periods=15).mean().plot(label='30yr average');
plt.legend();

As a result of the processing workflow the glacier directories can store many more files.

<div class="alert alert-warning">
    <b>Task:</b> Open the glacier directory of one glacier using your file explorer and take a look at its contents. During the practicals, we will build up a glacier directory from scratch to understand the purpose of each individual file.
</div>

You can find the location of a single glacier’s directory using the following command:

In [ ]:
gdir.dir

or you can print them directly as a list:

In [ ]:
import os
print(os.listdir(gdir.dir))

For a short explanation of what these files are, see the [glacier directory documentation](https://docs.oggm.org/en/stable/api.html#cfg-basenames).

### OGGM tasks

There are two different types of "[tasks](http://docs.oggm.org/en/stable/api.html#entity-tasks)" in OGGM:

**Entity Tasks**
: Standalone operations to be realized on one single glacier entity, independently from the others. The majority of OGGM tasks are entity tasks. They are parallelisable: the same task can run on several glaciers in parallel.

**Global Tasks**
: Tasks which require to work on several glacier entities at the same time. Model parameter calibration or the compilation of several glaciers' output are examples of global tasks. 
  
OGGM implements a simple mechanism to run a specific task on a list of `GlacierDirectory` objects. Here, for the sake of the demonstration of how tasks work, we are going to compute something new from the directory:

In [ ]:
from oggm import tasks

# run the compute_inversion_velocities task on all gdirs
workflow.execute_entity_task(tasks.compute_inversion_velocities, gdirs);

Compute inversion velocities is an optional task in the OGGM workflow. It computes the ice velocity along the flowline resulting from the ice flux from a glacier in equilibrium (more on this in the documentation or the dedicated [OGGM tutorial](https://tutorials.oggm.org/master/notebooks/tutorials/inversion.html)).

Note that OGGM tasks often do not return anything (the statement above seems to be "void", i.e. doing nothing). The vast majority of OGGM tasks actually write data to disk in order to retrieve it later. Let's have a look at the data we just added to the directory:

In [ ]:
inversion_output = gdir.read_pickle('inversion_output')  # The task above wrote the data to a pickle file - but we write to plenty of other files!

# Take the first flowline
fl = inversion_output[0]
# the last grid points often have irrealistic velocities
# because of the equilibrium assumption
vel = fl['u_surface'][:-1]  
plt.plot(vel, label='Velocity along flowline (m yr-1)'); plt.legend();

## Recap 

- OGGM is a "glacier centric" model, i.e. it operates on a list of glaciers.
- OGGM relies on saving files to disk during the workflow. Therefore, users must always specify a working directory while running OGGM.
- The working directory can be used to restart a run at a later stage.
- Put simply, this "restart workflow" is what OGGM uses to deliver data to its users. Pre-processed directories are online folders filled with glacier data that users can download.
- Once initialized locally, "glacier directories" allow OGGM to do what it does best: apply processing tasks to a list of glacier entities. These "entity tasks" get the data they need from disk, and write to disk once completed.
- Users (or subsequent OGGM tasks) can use these data for new computations.